In [132]:
import numpy as np
from pydrake.all import (
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LeafSystem,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    wrap_to,
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState, AcrobotParams, AcrobotSpongController
from underactuated import running_as_notebook

In [10]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [11]:
def UprightState():
    state = AcrobotState()
    state.set_theta1(np.pi)
    state.set_theta2(0.0)
    state.set_theta1dot(0.0)
    state.set_theta2dot(0.0)
    return state

In [ ]:
def BalancingLQR():

    acrobot = AcrobotPlant()
    context = acrobot.CreateDefaultContext()

    input = AcrobotInput()
    input.set_tau(0.0)
    acrobot.get_input_port(0).FixValue(context, input)

    context.get_mutable_continuous_state_vector().SetFromVector(
        UprightState().CopyToVector()
    )

    Q = np.diag((10.0, 10.0, 1.0, 1.0))
    R = [1]

    return LinearQuadraticRegulator(acrobot, context, Q, R)

In [112]:
class AcrobotEnergyShapingController(LeafSystem):
    def __init__(self, acrobot):
        super().__init__()
        self.DeclareVectorInputPort("state", 4)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        self.acrobot = acrobot
        self.acrobot_context = acrobot.CreateDefaultContext()
        self.SetAcrobotParams(AcrobotParams())

    def SetAcrobotParams(self, params):
        self.acrobot_context.get_mutable_numeric_parameter(0).SetFromVector(
            params.CopyToVector()
        )
        self.acrobot_context.SetContinuousState([np.pi, 0, 0, 0])
        self.desired_energy = (self.acrobot.EvalPotentialEnergy(self.acrobot_context) +
                               self.acrobot.EvalKineticEnergy(self.acrobot_context))

    def DoCalcOutput(self, context, output):
        acrobot_state = self.get_input_port(0).Eval(context)
        self.acrobot_context.SetContinuousState(acrobot_state)
        params = self.acrobot_context.get_numeric_parameter(0)
        
        theta2dot = acrobot_state[3]
        
        total_energy = (self.acrobot.EvalPotentialEnergy(self.acrobot_context) +
                        self.acrobot.EvalKineticEnergy(self.acrobot_context))
        energy_error = total_energy - self.desired_energy
        
        u = 0.1* theta2dot - 0.1 * theta2dot * energy_error
        u = np.clip(u, -5, 5)
        output.SetAtIndex(0, u)

In [124]:
class SwingUpAndBalanceAcrobotController(LeafSystem):
    def __init__(self, acrobot):
        # Initialize the LeafSystem.
        super().__init__()
        # Declare a four-dimensional state input port.
        self.DeclareVectorInputPort("state", 4)
        # Declare a one-dimensional control output port.
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        
        # Get the balancing LQR gains and S matrix.
        # We assume BalancingLQR is defined to work with an acrobot.
        (self.K, self.S) = BalancingLQR()
        
        # Create the energy shaping (swing-up) controller for the acrobot.
        # We assume AcrobotEnergyShapingController is defined similar to the pendulum version.
        self.energy_shaping = AcrobotEnergyShapingController(acrobot)
        self.energy_shaping_context = self.energy_shaping.CreateDefaultContext()


    def DoCalcOutput(self, context, output):
        # Read the current acrobot state ([theta1, theta2, theta1dot, theta2dot]).
        acrobot_state = self.get_input_port(0).Eval(context)
        # Copy the state for processing.
        # We use np.copy to avoid modifying the input state directly.
        xbar = np.copy(acrobot_state)
        
        # Wrap the angle states (assumed here to be the first two elements) so that they're near zero.
        xbar[0] = wrap_to(xbar[0], 0, 2.0 * np.pi) - np.pi
        xbar[1] = wrap_to(xbar[1], 0, 2.0 * np.pi) - np.pi
        
        # print(xbar.dot(self.S.dot(xbar)))
        if xbar.dot(self.S.dot(xbar)) < 100000.0:
            # The LQR control law is simply u = -K x.
            output.SetFromVector(-self.K.dot(xbar))
            print("Using LQR controller")
        else:
            # Otherwise, use the energy shaping controller for swing-up.
            # Pass the current acrobot state to the energy shaping controller.
            self.energy_shaping.get_input_port(0).FixValue(self.energy_shaping_context, acrobot_state)
            u_energy = self.energy_shaping.get_output_port(0).Eval(self.energy_shaping_context)
            output.SetFromVector(u_energy)
            print("Using Energy Shaping controller")

In [139]:
def acrobot_balancing_example():
    builder = DiagramBuilder()
    acrobot = builder.AddSystem(AcrobotPlant())

    saturation = builder.AddSystem(Saturation(min_value=[-7.5], max_value=[7.5]))
    builder.Connect(saturation.get_output_port(0), acrobot.get_input_port(0))
    wrapangles = WrapToSystem(4)
    wrapangles.set_interval(0, 0, 2.0 * np.pi)
    wrapangles.set_interval(1, -np.pi, np.pi)
    wrapto = builder.AddSystem(wrapangles)
    builder.Connect(acrobot.get_output_port(0), wrapto.get_input_port(0))
    controller = builder.AddSystem(AcrobotSpongController())
    builder.Connect(wrapto.get_output_port(0), controller.get_input_port(0))

    builder.Connect(controller.get_output_port(0), saturation.get_input_port(0))

    # Setup visualization
    scene_graph = builder.AddSystem(SceneGraph())
    AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    # Simulate
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    duration = 30.0 if running_as_notebook else 0.1
    for i in range(1):
        context.SetTime(0.0)
        context.SetContinuousState(
            UprightState().CopyToVector()*0
            + 0.02*np.array([1,3,0,0])
            
        )
        simulator.Initialize()
        simulator.AdvanceTo(duration)


acrobot_balancing_example()